In [1]:
# some parts of code is based on tutorial on Kmeans clustering discussed in the class......#
# Importing Libraries
from pyspark import SparkContext , SparkConf
from pyspark.mllib.clustering import KMeans,KMeansModel
from numpy import array
from math import sqrt
from pyspark.sql import Row
from pyspark.sql import SQLContext
conf=SparkConf().setMaster("local").setAppName("Kmeans")
sc
sql=SQLContext(sc)

In [2]:
# Reading data
lines=sc.textFile("xAPI-Edu-Data.csv")

In [3]:
# Removing Header
header=lines.first()
lines=lines.filter(lambda line:line !=header)
lines.take(5)

[u'M,KW,KuwaIT,lowerlevel,G-04,A,IT,F,Father,15,16,2,20,Yes,Good,Under-7,M',
 u'M,KW,KuwaIT,lowerlevel,G-04,A,IT,F,Father,20,20,3,25,Yes,Good,Under-7,M',
 u'M,KW,KuwaIT,lowerlevel,G-04,A,IT,F,Father,10,7,0,30,No,Bad,Above-7,L',
 u'M,KW,KuwaIT,lowerlevel,G-04,A,IT,F,Father,30,25,5,35,No,Bad,Above-7,L',
 u'M,KW,KuwaIT,lowerlevel,G-04,A,IT,F,Father,40,50,12,50,No,Bad,Above-7,M']

In [7]:
# Reading data by mapping to parseline function
data=lines.map(parseline)
data.take(5)

[Row(BithPlace='KuwaIT', Class='M', Nationality='KW', Topic='IT', absent='Under-7', announcements=2, discussion=20, gender='M', gradeid='G-04', raisehands=15, relation='Father', satisfaction='Good', sectionid='A', semester='F', stageid='lowerlevel', survey='Yes', visited=16),
 Row(BithPlace='KuwaIT', Class='M', Nationality='KW', Topic='IT', absent='Under-7', announcements=3, discussion=25, gender='M', gradeid='G-04', raisehands=20, relation='Father', satisfaction='Good', sectionid='A', semester='F', stageid='lowerlevel', survey='Yes', visited=20),
 Row(BithPlace='KuwaIT', Class='L', Nationality='KW', Topic='IT', absent='Above-7', announcements=0, discussion=30, gender='M', gradeid='G-04', raisehands=10, relation='Father', satisfaction='Bad', sectionid='A', semester='F', stageid='lowerlevel', survey='No', visited=7),
 Row(BithPlace='KuwaIT', Class='L', Nationality='KW', Topic='IT', absent='Above-7', announcements=5, discussion=35, gender='M', gradeid='G-04', raisehands=30, relation='Fat

In [6]:
#Function to read data in a CSV file
def parseline(line):
    field=line.split(',')
    return Row(gender=str(field[0]),
               Nationality=str(field[1]),
               BithPlace=str(field[2]),
               stageid=str(field[3]),
               gradeid=str(field[4]),
               sectionid=str(field[5]),
               Topic=str(field[6]),
              semester=str(field[7]),
               relation=str(field[8]),
               raisehands=int(field[9]),
               visited=int(field[10]),
               announcements=int(field[11]),
               discussion=int(field[12]),
               survey=str(field[13]),
               satisfaction=str(field[14]),
               absent=str(field[15].encode("utf-8")),
               Class=str(field[16])
               )
      

In [14]:
features = lines.map(lambda line: array([x for x in line.split(',')]))

In [15]:
params_only =  features.map(lambda x : array([int(x[9]),
                                                 int(x[10]),int(x[11]),int(x[12])]))

In [16]:
params_only

PythonRDD[12] at RDD at PythonRDD.scala:43

In [17]:
params_only.take(5)

[array([15, 16,  2, 20]),
 array([20, 20,  3, 25]),
 array([10,  7,  0, 30]),
 array([30, 25,  5, 35]),
 array([40, 50, 12, 50])]

In [46]:
for i in range(1,5):
    clusters = KMeans.train(params_only,i,maxIterations=100,runs=100,initializationMode="random")
    print(clusters)
    print (str(i)+ "clusters")

1clusters
2clusters
3clusters
4clusters


In [48]:
centroids=clusters.clusterCenters
print(" Cluster centers are " ,centroids)

(' Cluster centers are ', [array([ 76.4609375,  80.8984375,  66.484375 ,  73.625    ]), array([ 25.83783784,  31.78378378,  25.36486486,  64.74324324]), array([ 58.05263158,  77.70394737,  39.32894737,  25.49342105]), array([ 15.30952381,  14.16666667,  14.57142857,  21.31746032])])


In [ ]:
# from the results it is clear that  average no of raisedhands is 76.4 for a good student and 15.3 for poor student
#   average no of visitedResources is 76 for a good student and 15.3 for bad student
#   average no of Announcement Views is 66 for a good student and 15.3 for bad student
#   average no of Discussions is 73.625 for a good student and 21.31 for bad student